
# Assignment 3

This is a template notebook for Assignment 3.


## Install dependencies and initialization

In [ ]:
# The next 8 lines added to be compatible with version changes (2022/10/13).
# change cuda to 11.1
import os
p = os.getenv('PATH')
ld = os.getenv('LD_LIBRARY_PATH')
os.environ['PATH'] = f"/usr/local/cuda-11.3/bin:{p}"
os.environ['LD_LIBRARY_PATH'] = f"/usr/local/cuda-11.3/lib64:{ld}"
# change pytorch to 1.9.0 compiled with cuda 11.1
%pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113


# install dependencies:
%pip install pyyaml pycocotools>=2.0.1
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html
%pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
!pwd # shows current directory
!ls  # shows all files in this directory
!nvidia-smi # shows the specs and the current status of the allocated GPU

In [ ]:
# import some common libraries
from google.colab.patches import cv2_imshow
from sklearn.metrics import jaccard_score
from PIL import Image, ImageDraw
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import datetime
import random
import json
import cv2
import csv
import os

# import some common pytorch utilities
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch

# import some common detectron2 utilities
import detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
from detectron2.data import build_detection_test_loader
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
setup_logger()

<Logger detectron2 (DEBUG)>

In [ ]:
# Make sure that GPU is available for your notebook.
# Otherwise, you need to update the settungs in Runtime -> Change runtime type -> Hardware accelerator
torch.cuda.is_available()

In [ ]:
# You need to mount your google drive in order to load the data:
from google.colab import drive
drive.mount('/content/drive')
# Put all the corresponding data files in a data folder and put the data folder in a same directory with this notebook.
# Also create an output directory for your files such as the trained models and the output images.

In [ ]:
# Define the location of current directory, which should contain data/train, data/test, and data/train.json.
# TODO: approx 1 line
BASE_DIR = '/content/drive/My Drive/CMPT_CV_lab3'
OUTPUT_DIR = '{}/output'.format(BASE_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
os.chdir("/content/drive/My Drive/CMPT_CV_lab3")
!mkdir /localData
!cp -r data/test /localData/
!cp -r data/train /localData/
!cp data/train.json /localData/
!ls /localData

test  train  train.json


## Part 1: Object Detection

### Data Loader

In [ ]:
'''
# This function should return a list of data samples in which each sample is a dictionary.
# Make sure to select the correct bbox_mode for the data
# For the test data, you only have access to the images, therefore, the annotations should be empty.
# Other values could be obtained from the image files.
# TODO: approx 35 lines
'''
cIms = {}
addedToCache = []
localDir = "/localData"
def load_detection_data(set_name):
  data_dirs = '{}'.format(localDir)
  dataset = []
  added = []
  if set_name == 'train':
      json_file = os.path.join(data_dirs, 'train.json')
      with open(json_file) as f:
          imgs_anns = json.load(f)
      for idx, v in enumerate(imgs_anns):
        record = {}
        filename = os.path.join(data_dirs , "train", v["file_name"])


        obj = {
            "bbox": v["bbox"],
            "bbox_mode" : BoxMode.XYWH_ABS,
            "segmentation": v["segmentation"],
            "category_id": 0
        }
        if filename in added:
          i = added.index(filename)
          dataset[i]["annotations"].append(obj)
        else:
          img = cv2.imread(filename)
          height, width = img.shape[:2]
          record["file_name"] = filename
          added.append(filename)
          record["image_id"] = idx
          record["height"] = height
          record["width"] = width
          record["annotations"] = [obj]
          dataset.append(record)
  elif set_name == 'test':
    test_file = os.path.join(data_dirs, 'test')
    for idx, filename in enumerate(os.listdir(test_file)):
      record = {}
      fName = os.path.join(data_dirs, 'test', filename)
      img = cv2.imread(fName)
      height, width = img.shape[:2]
      record["file_name"] = fName
      record["image_id"] = idx
      record["height"] = height
      record["width"] = width
      record["annotations"] = []
      dataset.append(record)



  return dataset

testData = load_detection_data("test")
TVdata = load_detection_data("train")
train_len = round(0.7 * len(TVdata))
trainData, valData = TVdata[:train_len], TVdata[train_len:]

def get_detection_data(set_name):
  if set_name == "test":
    return testData
  elif set_name == "train":
    return trainData

  elif set_name == "val":
    return valData




In [ ]:
'''
# Remember to add your dataset to DatasetCatalog and MetadataCatalog
# Consdier "data_detection_train" and "data_detection_test" for registration
# You can also add an optional "data_detection_val" for your validation by spliting the training data
# TODO: approx 5 lines
'''
for d in ["train", "test", "val"]:
  DatasetCatalog.register("data_detection_" + d, lambda d=d: get_detection_data(d))
  MetadataCatalog.get("data_detection_" + d).set(thing_classes = ["planes"])
mData = MetadataCatalog.get("data_detection_train")



In [ ]:
# Visualize some samples using Visualizer to make sure that the function works correctly
# TODO: approx 5 lines
dataset_dicts = get_detection_data("train")
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])

    visualizer = Visualizer(img[:, :, ::-1], mData, scale=0.2)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])


### Set Configs

In [ ]:
'''
# Set the configs for the detection part in here.
# TODO: approx 15 lines
'''
cfg = get_cfg()
cfg.OUTPUT_DIR = "{}/output/".format(BASE_DIR)
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("data_detection_train",)
cfg.DATASETS.TEST = ()

#batch chnages to 4 reduce to 300 iterations
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 300
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

### Training

In [ ]:
#augmented data by normilizing by mean and standard deviation of data, also changed trainer from defult to detection trainer
class DetectionTrainer(DefaultTrainer):
    def train_loader(cfg):
        mapper = DatasetMapper(cfg, is_train=True, augmentations=[
            detectron2.data.Normalize(mean = [0.50867474, 0.5278123, 0.5297691],
                                      std = [0.22613746, 0.22742623, 0.2254186])
        ])
        return build_detection_train_loader(cfg, mapper=mapper)

In [ ]:
'''
# Create a DefaultTrainer using the above config and train the model
# TODO: approx 5 lines
'''
trainer = DetectionTrainer(cfg)
trainer.resume_or_load(resume = False)
trainer.train()


In [ ]:
%load_ext tensorboard
%tensorboard --logdir output

### Evaluation and Visualization

In [ ]:
'''
# After training the model, you need to update cfg.MODEL.WEIGHTS
# Define a DefaultPredictor
'''


cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, 'model_final.pth')
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6

predictor = DefaultPredictor(cfg)



[11/14 05:23:38 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/My Drive/CMPT_CV_lab3/output/model_final.pth ...


In [ ]:
'''
# Visualize the output for 3 random test samples
# TODO: approx 10 lines
'''
dataset_dicts = get_detection_data("test")
for d in random.sample(dataset_dicts, 3):
  img = cv2.imread(d["file_name"])

  outputs = predictor(img)
  visualizer = Visualizer(img[:,:,::-1],
                          metadata= mData,
                          scale = 0.2,
                          instance_mode=ColorMode.IMAGE_BW)
  out = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:,:,::-1])


In [ ]:
'''
# Use COCOEvaluator and build_detection_train_loader
# You can save the output predictions using inference_on_dataset
# TODO: approx 5 lines
'''
evaluator = COCOEvaluator("data_detection_val", output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, "data_detection_val")
print(inference_on_dataset(trainer.model, val_loader, evaluator))


### Improvements

For this part, you can bring any improvement which you have by adding new input parameters to the previous functions or defining new functions and variables.

In [ ]:
'''
# Bring any changes and updates regarding the improvement in here
'''


## Part 2: Semantic Segmentation

### Data Loader

In [ ]:
#cache the training data for speed
cIms.clear()
for t in trainData:
  if(t["file_name"] not in addedToCache):
    im = cv2.imread(t["file_name"])
    addedToCache.append(t["file_name"])
    cIms[t["file_name"]] = im


In [ ]:
'''
# Write a function that returns the cropped image and corresponding mask regarding the target bounding box
# idx is the index of the target bbox in the data
# high-resolution image could be passed or could be load from data['file_name']
# You can use the mask attribute of detectron2.utils.visualizer.GenericMask
#     to convert the segmentation annotations to binary masks
# TODO: approx 10 lines
'''

from detectron2.utils.visualizer import GenericMask

def get_instance_sample(data, idx, img=None):
  height = data["height"]
  width = data["width"]

  annos = data["annotations"][idx]

  bx, by, bw, bh = annos["bbox"]
  segMask = annos["segmentation"]
  if (img == None):
    img = cIms[data["file_name"]]
  obj_img = img[round(by): round(by + bh), round(bx): round(bx + bw)]
  obj_mask = np.array(GenericMask(segMask, height, width).mask)
  obj_mask = obj_mask[round(by): round(by + bh), round(bx): round(bx + bw)]

  return obj_img, obj_mask

#version to use for part 3 since the version for part 2 relies on the caching of the training set
def get_instance_sample2(data, idx, img=None):
  height = data["height"]
  width = data["width"]

  annos = data["annotations"][idx]

  bx, by, bw, bh = annos["bbox"]
  segMask = annos["segmentation"]
  if (img == None):
    img = cv2.imread(data["file_name"])
  obj_img = img[round(by): round(by + bh), round(bx): round(bx + bw)]
  obj_mask = np.array(GenericMask(segMask, height, width).mask)
  obj_mask = obj_mask[round(by): round(by + bh), round(bx): round(bx + bw)]

  return obj_img, obj_mask



In [ ]:
'''
# We have provided a template data loader for your segmentation training
# You need to complete the __getitem__() function before running the code
# You may also need to add data augmentation or normalization in here
'''

class PlaneDataset(Dataset):
  def __init__(self, set_name, data_list):
      self.transforms = transforms.Compose([
          transforms.ToTensor(), # Converting the image to tensor and change the image format (Channels-Last => Channels-First)
      ])
      self.set_name = set_name
      self.data = data_list
      self.instance_map = []
      for i, d in enumerate(self.data):
        for j in range(len(d['annotations'])):
          self.instance_map.append([i,j])

  '''
  # you can change the value of length to a small number like 10 for debugging of your training procedure and overfeating
  # make sure to use the correct length for the final training
  '''
  def __len__(self):
      return len(self.instance_map)

  def numpy_to_tensor(self, img, mask):
    if self.transforms is not None:
        img = self.transforms(img)
    img = torch.tensor(img, dtype=torch.float)
    mask = torch.tensor(mask, dtype=torch.float)
    return img, mask.unsqueeze(0)


  '''
  # Complete this part by using get_instance_sample function
  # make sure to resize the img and mask to a fixed size (for example 128*128)
  # you can use "interpolate" function of pytorch or "numpy.resize"
  # TODO: 5 lines
  '''
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
        idx = idx.tolist()
    idx = self.instance_map[idx]
    data = self.data[idx[0]]
    #instance map has index of data, index of annotation
    img, mask = get_instance_sample(data, idx[1])

    img = cv2.resize(img, dsize=(128, 128), interpolation=cv2.INTER_NEAREST)
    mask = cv2.resize(mask, dsize=(128, 128), interpolation=cv2.INTER_NEAREST)

    img, mask = self.numpy_to_tensor(img, mask)




    return img, mask

def get_plane_dataset(set_name='train', batch_size=2):
    my_data_list = DatasetCatalog.get("data_detection_{}".format(set_name))
    dataset = PlaneDataset(set_name, my_data_list)
    loader = DataLoader(dataset, batch_size=batch_size, num_workers=4,
                                              pin_memory=True, shuffle=True)
    return loader, dataset

### Network

In [ ]:
'''
# convolution module as a template layer consists of conv2d layer, batch normalization, and relu activation
'''
class conv(nn.Module):
    def __init__(self, in_ch, out_ch, activation=True):
        super(conv, self).__init__()
        if(activation):
          self.layer = nn.Sequential(
             nn.Conv2d(in_ch, out_ch, 3, padding=1),
             nn.BatchNorm2d(out_ch),
             nn.ReLU(inplace=True)
          )
        else:
          self.layer = nn.Sequential(
             nn.Conv2d(in_ch, out_ch, 3, padding=1)  ,
             )

    def forward(self, x):
        x = self.layer(x)
        return x


'''
# downsampling module equal to a conv module followed by a max-pool layer
'''
class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.layer = nn.Sequential(
            conv(in_ch, out_ch),
            nn.MaxPool2d(2)
            )

    def forward(self, x):
        x = self.layer(x)
        return x

'''
# upsampling module equal to a upsample function followed by a conv module
'''
class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=False):
        super(up, self).__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch, in_ch, 2, stride=2)

        self.conv = conv(in_ch, out_ch)

    def forward(self, x):
        y = self.up(x)
        y = self.conv(y)
        return y

'''
# the main model which you need to complete by using above modules.
# you can also modify the above modules in order to improve your results.
'''
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        # Encoder

        self.input_conv = conv(3, 8)
        self.down_1 = down(8, 16)
        self.down_2 = down(16, 32)
        self.down_3 = down(32, 64)
        self.down_4 = down(64, 128)
        self.down_5 = down(128, 256)
        self.down_6 = down(256, 512)

        # Decoder

        self.up_1 = up(512, 256)
        self.up_2 = up(256, 128)
        self.up_3 = up(128, 64)
        self.up_4 = up(64, 32)
        self.up_5 = up(32, 16)
        self.up_6 = up(16, 8)
        self.up_7 = conv(8, 3)
        self.output_conv = conv(3, 1, False) # ReLu activation is removed to keep the logits for the loss function

        self.normal_1 = nn.BatchNorm2d(8)
        self.normal_2 = nn.BatchNorm2d(16)
        self.normal_3 = nn.BatchNorm2d(32)
        self.normal_4 = nn.BatchNorm2d(64)
        self.normal_5 = nn.BatchNorm2d(128)
        self.normal_6 = nn.BatchNorm2d(256)

    def forward(self, input):
      y_1 = self.input_conv(input)
      y_2 = self.down_1(y_1)
      y_3 = self.down_2(y_2)
      y_4 = self.down_3(y_3)
      y_5 = self.down_4(y_4)
      y_6 = self.down_5(y_5)
      y_7 = self.down_6(y_6)
      y_7 = self.normal_6(F.relu(self.up_1(y_7) + y_6))
      y_6 = self.normal_5(F.relu(self.up_2(y_7) + y_5))
      y_5 = self.normal_4(F.relu(self.up_3(y_6) + y_4))
      y_4 = self.normal_3(F.relu(self.up_4(y_5) + y_3))
      y_3 = self.normal_2(F.relu(self.up_5(y_4) + y_2))
      y_2 = self.normal_1(F.relu(self.up_6(y_3) + y_1))
      y = self.up_7(y_2)
      output = self.output_conv(y)
      return output

### Training

In [ ]:
'''
# The following is a basic training procedure to train the network
# You need to update the code to get the best performance
# TODO: approx ? lines
'''
import matplotlib.pyplot as plt


# Set the hyperparameters
num_epochs = 10
batch_size = 6
learning_rate = 0.01
weight_decay = 1e-5

model = MyModel() # initialize the model
model = model.cuda() # move the model to GPU
loader, _ = get_plane_dataset('train', batch_size) # initialize data_loader
crit = nn.BCEWithLogitsLoss() # Define the loss function
optim = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay) # Initialize the optimizer as SGD

# start the training procedure
losses = []
lenData = len(loader)
for epoch in range(num_epochs):
  total_loss = 0
  for (img, mask) in tqdm(loader):
    img = torch.tensor(img, device=torch.device('cuda'), requires_grad = True)
    mask = torch.tensor(mask, device=torch.device('cuda'), requires_grad = True)
    pred = model(img)
    loss = crit(pred, mask)
    optim.zero_grad()
    loss.backward()
    optim.step()
    total_loss += loss.cpu().data * img.size(0)
  epoch_loss = (total_loss / lenData)
  losses.append(epoch_loss)
  print("Epoch: {}, Loss: {}".format(epoch, total_loss/lenData))
  torch.save(model.state_dict(), '{}/output/{}_segmentation_model.pth'.format(BASE_DIR, epoch))
plt.plot(losses)
'''
# Saving the final model
'''
torch.save(model.state_dict(), '{}/output/final_segmentation_model.pth'.format(BASE_DIR))


### Evaluation and Visualization

In [ ]:
'''
# Before starting the evaluation, you need to set the model mode to eval
# You may load the trained model again, in case if you want to continue your code later
# TODO: approx 15 lines
'''
batch_size = 8
model = MyModel().cuda()
model.load_state_dict(torch.load('{}/output/final_segmentation_model.pth'.format(BASE_DIR)))
model = model.eval() # chaning the model to evaluation mode will fix the bachnorm layers
loader, dataset = get_plane_dataset('train', batch_size)
import math
def calcIou(pred, mask):
  SMOOTH = 1 * math.e - 6
  mask = mask.squeeze(1).int().cpu().numpy()
  pred = pred.squeeze(1)
  pred = torch.sigmoid(pred).cpu().numpy()
  pred = (pred >= 0.5)

  intersection = (pred & mask)
  intersection = intersection.sum((1, 2))

  union = (pred|mask)
  union = union.sum((1, 2))

  iou = (intersection + SMOOTH) / (union + SMOOTH)
  return iou.mean()
total_iou = 0
numChecked = 0
for (img, mask) in tqdm(loader):
  with torch.no_grad():
    img = img.cuda()
    mask = mask.cuda()
    pred = model(img)
    total_iou = total_iou + calcIou(pred, mask)
    numChecked+=1


    '''
    ## Complete the code by obtaining the IoU for each img and print the final Mean IoU
    '''


print("\n #images: {}, Mean IoU: {}".format(len(dataset), total_iou/numChecked))


In [ ]:
'''
# Visualize 3 sample outputs
# TODO: approx 5 lines
'''
batch_size = 4
loader, dataset = get_plane_dataset('train', batch_size)


img, mask = next(iter(loader))
with torch.no_grad():
    img = img.cuda()
    mask = mask.cpu().detach()
    pred = model(img).cpu().detach()
    for i in range(img.shape[0]):
        newimg = transforms.ToPILImage()(img[i].cpu())
        cv2_imshow(np.array(newimg))
        predicted = np.rint(torch.sigmoid((pred[i])[0]))*255
        masked = np.array(mask[i])[0]*255
        predicted = np.array(predicted)
        cv2_imshow(predicted)
        cv2_imshow(masked)



## Part 3: Instance Segmentation

In this part, you need to obtain the instance segmentation results for the test data by using the trained segmentation model in the previous part and the detection model in Part 1.

### Get Prediction

In [ ]:
'''
# Define a new function to obtain the prediction mask by passing a sample data
# For this part, you need to use all the previous parts (predictor, get_instance_sample, data preprocessings, etc)
# It is better to keep everything (as well as the output of this funcion) on gpu as tensors to speed up the operations.
# pred_mask is the instance segmentation result and should have different values for different planes.
# TODO: approx 35 lines
'''

model = MyModel().cuda()
model.load_state_dict(torch.load("{}/output/final_segmentation_model.pth".format(BASE_DIR)))

def get_prediction_mask(data):
  img = cv2.imread(data["file_name"])
  imW = data["width"]
  imH = data["height"]

  ten_transforms = transforms.Compose([
        transforms.ToTensor(),                                          # Converting the image to tensor and change the image format (Channels-Last => Channels-First)
    ])

  pred_mask = np.zeros((imH, imW))
  gt_mask = np.zeros((imH, imW))

  if data["annotations"]:
    for i in range(len(data["annotations"])):
      bboxData = data["annotations"][i]["bbox"]
      bx, by, bw, bh = [round(c) for c in bboxData]

      bbox, temp_mask = get_instance_sample(data, i, img=None)
      #different value for segmentation
      gt_mask[by:by+bh, bx:bx+bw] = (i+1) * temp_mask
      bboxPreds = predictor(img)
      boxes = bboxPreds["instances"].pred_boxes
    for idx, b in enumerate(boxes):
      x1, y1, x2, y2 = [round(item) for item in b.cpu().numpy().tolist()]
      cropped = img[y1:y2, x1:x2, :]
      cropped = cv2.resize(cropped, (128,128),interpolation = cv2.INTER_NEAREST)
      cropped = ten_transforms(cropped).cuda()

      temp_pred = torch.sigmoid(model(cropped.unsqueeze(0))).squeeze(1).squeeze(0)
      temp_pred = temp_pred.detach().cpu().numpy()
      #resize to fix bounding box again
      temp_pred = cv2.resize(temp_pred, (x2-x1, y2-y1), interpolation=cv2.INTER_NEAREST)
      #need to threshold
      temp_pred = np.array(temp_pred >= 0.5, dtype = np.int64)
      pred_mask[y1:y2, x1:x2] = (idx+1) * temp_pred
  else:
      bboxPreds = predictor(img)
      boxes = bboxPreds["instances"].pred_boxes

      for idx, b, in enumerate(boxes):
        #uses XYXY coords
        x1, y1, x2, y2 = [round(item) for item in b.cpu().numpy().tolist()]
        cropped = img[y1:y2, x1:x2, :]
        cropped = cv2.resize(cropped, (128,128),interpolation = cv2.INTER_NEAREST)
        cropped = ten_transforms(cropped).cuda()

        temp_pred = torch.sigmoid(model(cropped.unsqueeze(0))).squeeze(1).squeeze(0)
        temp_pred = temp_pred.detach().cpu().numpy()
        #resize to fix bounding box again
        temp_pred = cv2.resize(temp_pred, (x2-x1, y2-y1), interpolation=cv2.INTER_NEAREST)
        #need to threshold
        temp_pred = np.array(temp_pred >= 0.5, dtype = np.int64)
        pred_mask[y1:y2, x1:x2] = (idx+1) * temp_pred






  return img, gt_mask, torch.tensor(pred_mask, device = 'cuda') # gt_mask could be all zero when the ground truth is not given.


### Visualization and Submission

In [ ]:
'''
# Visualise the output prediction as well as the GT Mask and Input image for a sample input
# TODO: approx 10 lines
'''
import matplotlib.pyplot as plt
fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(20, 15))
fig.suptitle("Visualizing test set", fontsize=16)

axs[0][0].set_title("Image")
axs[0][1].set_title("Segmentation")
axs[0][2].set_title("Prediction")

random_test = [item for item in random.sample(DatasetCatalog.get("data_detection_test"), 3)]

for idx in range(3):
    img, gt_mask, pred_mask = get_prediction_mask(random_test[idx])

    axs[idx][0].imshow(img)
    axs[idx][1].imshow(gt_mask)
    axs[idx][2].imshow(pred_mask.cpu())

    axs[idx][0].set_xticks([])
    axs[idx][0].set_yticks([])
    axs[idx][1].set_xticks([])
    axs[idx][1].set_yticks([])
    axs[idx][2].set_xticks([])
    axs[idx][2].set_yticks([])



In [ ]:
'''
# ref: https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
# https://www.kaggle.com/c/airbus-ship-detection/overview/evaluation
'''
def rle_encoding(x):
    '''
    x: pytorch tensor on gpu, 1 - mask, 0 - background
    Returns run length as list
    '''
    dots = torch.where(torch.flatten(x.long())==1)[0]
    if(len(dots)==0):
      return []
    inds = torch.where(dots[1:]!=dots[:-1]+1)[0]+1
    inds = torch.cat((torch.tensor([0], device=torch.device('cuda'), dtype=torch.long), inds))
    tmpdots = dots[inds]
    inds = torch.cat((inds, torch.tensor([len(dots)], device=torch.device('cuda'))))
    inds = inds[1:] - inds[:-1]
    runs = torch.cat((tmpdots, inds)).reshape((2,-1))
    runs = torch.flatten(torch.transpose(runs, 0, 1)).cpu().data.numpy()
    return ' '.join([str(i) for i in runs])

In [ ]:
'''
# You need to upload the csv file on kaggle
# The speed of your code in the previous parts highly affects the running time of this part
'''

preddic = {"ImageId": [], "EncodedPixels": []}

'''
# Writing the predictions of the training set
'''
my_data_list = DatasetCatalog.get("data_detection_{}".format('train'))
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample)
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for index in inds:
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index)
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)
#clear training stuff out of cache
cIms.clear()
#cache the val data for speed
for t in valData:
  if(t["file_name"] not in addedToCache):
    im = cv2.imread(t["file_name"])
    addedToCache.append(t["file_name"])
    cIms[t["file_name"]] = im
#predictions for val set
my_data_list = DatasetCatalog.get("data_detection_{}".format('val'))
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample)
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for index in inds:
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index)
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)


'''
# Writing the predictions of the test set
'''

my_data_list = DatasetCatalog.get("data_detection_{}".format('test'))
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample)
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for j, index in enumerate(inds):
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index).double()
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

pred_file = open("{}/pred.csv".format(BASE_DIR), 'w')
pd.DataFrame(preddic).to_csv(pred_file, index=False)
pred_file.close()


## Part 4: Mask R-CNN

For this part you need to follow a same procedure to part 2 with the configs of Mask R-CNN, other parts are generally the same as part 2.

### Data Loader

In [ ]:
#reuse the data loader from the previous parts
cIms.clear()

### Network

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("data_detection_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0025  # pick a good LR
cfg.SOLVER.MAX_ITER = 500    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
cfg.OUTPUT_DIR = f"{BASE_DIR}/output/"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

### Training

In [ ]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

### Evaluation and Visualization

In [ ]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_detection_data("val")
for d in random.sample(dataset_dicts, 3):
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=mData,
                   scale=0.2,
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("data_detection_val", cfg, False, output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, "data_detection_val")
print(inference_on_dataset(trainer.model, val_loader, evaluator))